In [1]:
import pandas as pd
import numpy as np


data = pd.read_csv('InputedData.csv')
y = data['RainTomorrow']
date = data['Date']
data = data.drop(['Date', 'RainTomorrow'], axis = 1)


X_train = pd.DataFrame(columns = data.columns)
y_train = pd.Series(dtype='int64')
X_test = pd.DataFrame(columns = data.columns)
y_test = pd.Series(dtype='int64')


for j in range(16, 65):
    location_labels = np.where(data.iloc[: ,j] == 1)[0]
    last_train = int(0.75 * len(location_labels))
    X_train = X_train.append(data.iloc[location_labels[0:last_train], :])
    y_train = y_train.append(y[location_labels[0:last_train]])
    X_test = X_test.append(data.iloc[location_labels[last_train:], :])
    y_test = y_test.append(y[location_labels[last_train:]])
    
    
X_train.to_csv("X_train.csv")
X_test.to_csv('X_test.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')




In [7]:
# Read csv. Desert the first column which is the index
X_train, X_test, y_train, y_test = pd.read_csv('X_train.csv').iloc[: , 1:], \
                                   pd.read_csv('X_test.csv').iloc[: , 1:],\
                                   pd.read_csv('y_train.csv').iloc[: , 1:], \
                                   pd.read_csv('y_test.csv').iloc[: , 1:]  



L = 10 # The length of the time series for each sample


import torch
from torch import nn

def CreateRainfullDataSet_RNN(X, y, L):
    input_size = X.shape[1]
    X_RNN = torch.empty((1, L, input_size), dtype=torch.float64)
    y_RNN = torch.empty(1, dtype=torch.long)
    for j in range(16, 65):
        location_labels = np.where(X.iloc[: ,j] == 1)[0]
        training_index = []
        for i in location_labels[(L - 1):len(location_labels)]:
            training_index += np.arange(i- L + 1, i + 1).tolist()
            
        
        X_RNN  = torch.cat((X_RNN, torch.tensor(X.iloc[training_index, :].values).view(-1, L, input_size)), 0)
        y_RNN  = torch.cat((y_RNN, torch.tensor(y.iloc[location_labels[(L - 1):len(location_labels)]].values).view(-1)))
            
    return X_RNN[1:, :].float(), y_RNN[1:]

X_train_RNN, y_train_RNN = CreateRainfullDataSet_RNN(X_train, y_train, L)
X_test_RNN, y_test_RNN = CreateRainfullDataSet_RNN(X_test, y_test, L)